# Clone repo & Install dependencies

In [1]:
!git clone -b gradio https://github.com/RSWAIN1486/emlov3-pytorchlightning-hydra.git

Cloning into 'emlov3-pytorchlightning-hydra'...
remote: Enumerating objects: 450, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 450 (delta 3), reused 0 (delta 0), pack-reused 436
Receiving objects: 100% (450/450), 3.22 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [2]:
!mv emlov3-pytorchlightning-hydra pl-hy-dev
%cd pl-hy-dev

!pip install --quiet -r requirements.txt
!pip install --quiet -e .

/content/pl-hy-dev
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Run Training and save as Torch Traced model


### Best Hyperparams and val_loss
```
learning_rate : 0.004
batch_size : 1024
datamodule.block_size : 8
model.net.n_embed : 256
model.net.n_heads : 8
model.net.drop_p : 0.15
model.net.n_decoder_blocks : 4
Best val loss : 3.8065
```





In [1]:
%cd /content/pl-hy-dev
!src_train -m experiment=harrypotter_jit.yaml test=False trainer.max_epochs=10 trainer.accelerator=gpu save_torchtrace=True paths.ckpt_jittrace_save_path=ckpt/gpt_torch_traced.pt

Streaming output truncated to the last 5000 lines.
                                                               0.491 train/loss:
Epoch 9/9  ━━━━━━━━━━━━━━━━━ 0/1176 0:00:58 • -:--:-- 0.00it/s v_num: 0         
                                                               val/loss: 0.491  
                                                               val/loss_best:   
                                                               0.491 train/loss:
Epoch 9/9  ━━━━━━━━━━━━━━━━━ 0/1176 0:00:59 • -:--:-- 0.00it/s v_num: 0         
                                                               val/loss: 0.491  
                                                               val/loss_best:   
                                                               0.491 train/loss:
Epoch 9/9  ━━━━━━━━━━━━━━━━━ 0/1176 0:01:00 • -:--:-- 0.00it/s v_num: 0         
                                                               val/loss: 0.491  
                                                          

In [ ]:
from google.colab import files

try:
  files.download("/content/pl-hy-dev/ckpt/gpt_torch_traced.pt")
except Exception as e:
  print(e)

# Gradio Demo to generate Harry Potter text using traced model

In [ ]:
!pip install gradio

In [ ]:
import tiktoken
import torch
import gradio as gr


def get_encoder():
  cl100k_base = tiktoken.get_encoding("cl100k_base")

  # In production, load the arguments directly instead of accessing private attributes
  # See openai_public.py for examples of arguments for specific encodings
  encoder = tiktoken.Encoding(
      # If you're changing the set of special tokens, make sure to use a different name
      # It should be clear from the name what behaviour to expect.
      name="cl100k_im",
      pat_str=cl100k_base._pat_str,
      mergeable_ranks=cl100k_base._mergeable_ranks,
      special_tokens={
          **cl100k_base._special_tokens,
          "<|im_start|>": 100264,
          "<|im_end|>": 100265,
            }
        )

  return encoder

def text_completion(text: str) -> str:

  # encoder
  encoder = get_encoder()

  input_enc = torch.tensor(encoder.encode(text))
  with torch.no_grad():
      out_gen = loaded_model.model.generate(input_enc.unsqueeze(0).long(), max_new_tokens=256)
  decoded = encoder.decode(out_gen[0].cpu().numpy().tolist())
  return decoded

In [ ]:
# model
loaded_model = torch.jit.load("/content/pl-hy-dev/ckpt/gpt_torch_traced.pt", map_location="cpu")
loaded_model.eval()


In [ ]:
demo = gr.Interface(
    fn=text_completion,
    inputs=gr.Textbox(lines=5, placeholder="Enter text here..."),
    outputs="text",
)

In [ ]:
demo.launch(share=True, debug=True)